In [14]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import (GridSearchCV, cross_validate,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from get_data import ceramiche_DB

In [18]:
h = .02  # step size in the mesh

classifiers = {
    "Nearest Neighbors": (
                    KNeighborsClassifier(3),
                    {'n_neighbors': list(range(1:11)), 'weights': ['uniform', 'distance']}
                ),
    # "Linear SVM": (
    #                 SVC(kernel="linear", C=0.025),
    #                 {}
    #             ),
    "Decision Tree": (
                    DecisionTreeClassifier(max_depth=5),
                    {'criterion': ['gini', 'entropy']}
                ),
    "Random Forest": (
                    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
                    {}
                ),
    "Naive Bayes": (
                    GaussianNB(),
                    {}
                ),


    # "RBF SVM": SVC(gamma=2, C=1),
    # "Gaussian Process": GaussianProcessClassifier(1.0 * RBF(1.0)),
    # "Neural Net": MLPClassifier(alpha=1, max_iter=1000),
    # "AdaBoost": AdaBoostClassifier(),
    # "QDA": QuadraticDiscriminantAnalysis(),
}

In [17]:
def sensibility(y_true, y_predict):
    c_matrix = confusion_matrix(y_true, y_predict)
    return c_matrix[1][1]/sum(c_matrix[1])


def specificity(y_true, y_predict):
    c_matrix = confusion_matrix(y_true, y_predict)
    return c_matrix[0][0]/sum(c_matrix[0])


scores = {
    'sensibility' : make_scorer(sensibility, greater_is_better=True),
    'specificity' : make_scorer(specificity, greater_is_better=True),
    'accuracy' : make_scorer(accuracy_score, greater_is_better=True)
}